<a href="https://colab.research.google.com/github/Jaejuna/SpeedWagon/blob/main/KoBART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/SKT-AI/KoBART#how-to-install

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 126.2 MB/s eta 0:00:00


In [3]:
import torch
import transformers
import tensorflow

In [4]:
!git clone https://github.com/SKT-AI/KoBART.git

Cloning into 'KoBART'...
remote: Enumerating objects: 331, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 331 (delta 13), reused 14 (delta 13), pack-reused 303
Receiving objects: 100% (331/331), 8.24 MiB | 16.51 MiB/s, done.
Resolving deltas: 100% (176/176), done.


In [ ]:
%pwd

'/content'

In [9]:
%cd KoBART

/content/KoBART


In [ ]:
torch.__version__

'2.0.1+cu118'

In [ ]:
pip uninstall torch

Found existing installation: torch 2.0.1+cu118
Uninstalling torch-2.0.1+cu118:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.10/dist-packages/functorch/*
    /usr/local/lib/python3.10/dist-packages/nvfuser/*
    /usr/local/lib/python3.10/dist-packages/torch-2.0.1+cu118.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch/*
    /usr/local/lib/python3.10/dist-packages/torchgen/*
Proceed (Y/n)? Y
  Successfully uninstalled torch-2.0.1+cu118


In [ ]:
pip install torch==1.7.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torch==1.7.1 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.7.1


In [10]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.2/814.2 kB 17.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement torch==1.7.1 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.7.1


In [6]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.3 MB/s eta 0:00:00


In [14]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.7 MB/s eta 0:00:00


In [11]:
from kobart import get_kobart_tokenizer
kobart_tokenizer = get_kobart_tokenizer()
kobart_tokenizer.tokenize("안녕하세요. 한국어 BART 입니다.🤣:)l^o")

/content/KoBART/.cache/kobart_base_tokenizer_cased_cf74400bce.zip[██████████████████████████████████████████████████]


['▁안녕하', '세요.', '▁한국어', '▁B', 'A', 'R', 'T', '▁입', '니다.', '🤣', ':)', 'l^o']

In [12]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/BOAZ_miniProject2/train.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/BOAZ_miniProject2/valid.csv')

In [16]:
import datasets
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import itertools
import re
import torch
from rouge import Rouge
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback
)
from datasets import Dataset
from tqdm import tqdm
import glob
import json
import os
     

ModuleNotFoundError: ignored

In [15]:
# DataFrame을 Dataset으로 전환
import datasets
train_data = Dataset.from_pandas(train_df) 
val_data = Dataset.from_pandas(val_df)
test_samples = Dataset.from_pandas(val_df)

print(train_data)
print(val_data)
print(test_samples)

NameError: ignored

In [17]:
from transformers import BartForConditionalGeneration
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer
kobart_tokenizer = get_kobart_tokenizer()
model = BartForConditionalGeneration.from_pretrained(get_pytorch_kobart_model())
text = """
1일 오후 9시까지 최소 20만3220명이 코로나19에 신규 확진됐다. 또다시 동시간대 최다 기록으로, 사상 처음 20만명대에 진입했다.
방역 당국과 서울시 등 각 지방자치단체에 따르면 이날 0시부터 오후 9시까지 전국 신규 확진자는 총 20만3220명으로 집계됐다.
국내 신규 확진자 수가 20만명대를 넘어선 것은 이번이 처음이다.
동시간대 최다 기록은 지난 23일 오후 9시 기준 16만1389명이었는데, 이를 무려 4만1831명이나 웃돌았다. 전날 같은 시간 기록한 13만3481명보다도 6만9739명 많다.
확진자 폭증은 3시간 전인 오후 6시 집계에서도 예견됐다.
오후 6시까지 최소 17만8603명이 신규 확진돼 동시간대 최다 기록(24일 13만8419명)을 갈아치운 데 이어 이미 직전 0시 기준 역대 최다 기록도 넘어섰다. 역대 최다 기록은 지난 23일 0시 기준 17만1451명이었다.
17개 지자체별로 보면 서울 4만6938명, 경기 6만7322명, 인천 1만985명 등 수도권이 12만5245명으로 전체의 61.6%를 차지했다. 서울과 경기는 모두 동시간대 기준 최다로, 처음으로 각각 4만명과 6만명을 넘어섰다.
비수도권에서는 7만7975명(38.3%)이 발생했다. 제주를 제외한 나머지 지역에서 모두 동시간대 최다를 새로 썼다.
부산 1만890명, 경남 9909명, 대구 6900명, 경북 6977명, 충남 5900명, 대전 5292명, 전북 5150명, 울산 5141명, 광주 5130명, 전남 4996명, 강원 4932명, 충북 3845명, 제주 1513명, 세종 1400명이다.
집계를 마감하는 자정까지 시간이 남아있는 만큼 2일 0시 기준으로 발표될 신규 확진자 수는 이보다 더 늘어날 수 있다. 이에 따라 최종 집계되는 확진자 수는 21만명 안팎을 기록할 수 있을 전망이다.
한편 전날 하루 선별진료소에서 이뤄진 검사는 70만8763건으로 검사 양성률은 40.5%다. 양성률이 40%를 넘은 것은 이번이 처음이다. 확산세가 계속 거세질 수 있다는 얘기다.
이날 0시 기준 신규 확진자는 13만8993명이었다. 이틀 연속 13만명대를 이어갔다.
"""
text = text.replace('\n', ' ')

inputs = kobart_tokenizer(text, truncation=True, padding="longest", return_tensors="pt")
raw_input_ids = kobart_tokenizer.encode(text)
input_ids = [kobart_tokenizer.bos_token_id] + raw_input_ids + [kobart_tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
kobart_tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

using cached model. /content/KoBART/.cache/kobart_base_tokenizer_cased_cf74400bce.zip
/content/KoBART/.cache/kobart_base_cased_ff4bda5738.zip[██████████████████████████████████████████████████]


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'"다.다.다.다.다.다.다. 최소 20만3220명이 코로나19에 신규 확진됐다. 또다시 동시간대 최다 기록으로, 사상 처음 20만명대에 진입했다. 방역 당국과 서울시 등 각 지방자치단체에 따르면 이날 0시부터 오후 9시까지 전국 신규 확진자는 총 20만3220명으로 집계됐다. 국내 신규 확진자 수가 20만명대를 넘어선 것은 이번이 처음이다. 동시간대 최다 기록은 지난 23일 오후 9시 기준 16만1389명이었는데, 이를 무려 4만1831명이나 웃돌았다. 전날 같은 시간 기록한 13만3481명보다도 6만9739명이나 웃돌았다. 전날 같은 시간 기록한 13만3481명보다도 6만9739명 많다. 확진자 폭증은 3시간 전인 오후 6시 집계에서도 예견됐다. 오후 6시까지 최소 17만8603명이 신규 확진돼 동시간대 최다 기록(24일 13만8419명)을 갈아치운 데 이어 이미 직전 0시 기준 역대 최다 기록도 넘어섰다. 역대 최다 기록은 지난 23일 0시 기준 17만1451명이었다. 17개 지자체별로 보면 서울 4만6938명, 경기 6만7322명, 인천 1만985명 등 수도권이 12만5245명으로 전체의 61.6%를 차지했다. 서울과 경기는 모두 동시간대 기준 최다로, 처음으로 각각 4만명과 6만명을 넘어섰다. 비수도권에서는 7만7975명(38.3%)이 발생했다. 제주를 제외한 나머지 지역에서 모두 동시간대 최다를 새로 썼다. 부산 1만890명, 경남 9909명, 대구 6900명, 경북 6977명, 충남 5900명, 충남 5900명, 대전 5292명, 전북 5150명, 울산 5141명, 대구 5130명, 경북 4996명, 전남 4996명, 강원 4932명, 광주 3845명, 전남 4996명, 강원 4932명, 충북 3845명, 제주 1513명, 세종 1400명이다. 집계를 마감하는 자정까지 시간이 남아있는 만큼 2일 0시 기준으로 발표될 신규 확진자 수는 이보다 더 늘어날 수 있다. 이에 따라 최종 집계되는 확진자 수는 21만명 안팎을 기록할 수 있을 전망이다. 한편 

In [ ]:
train.iloc[0,0]

'웃긴게 20년동안... 바로 옆 가게 주차장에서 같이 그 감자탕집 주차도 했었는데 그 가게에서 이제 못대게해서; 졸지에...감자탕집 주차 없어짐 ㅜㅜ 헐 의상하는 일 있었나? 뭐..아무래도 그 감자탕집이 장사 잘되니까 ㅋㅋㅋㅋㅋㅋㅋ 배아파서.. 아.. 그렇다고 감자탕집을 안간다해서 그 가게는 안갈거같은데; 거기는..뭔 능이버섯 오리전골? 뭐 이런거라 대중적이진 않잖아 아 점심으로 투머치..'

In [ ]:
from transformers import BartForConditionalGeneration
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer
kobart_tokenizer = get_kobart_tokenizer()
model = BartForConditionalGeneration.from_pretrained(get_pytorch_kobart_model())

using cached model. /content/KoBART/.cache/kobart_base_tokenizer_cased_cf74400bce.zip
/content/KoBART/.cache/kobart_base_cased_ff4bda5738.zip[██████████████████████████████████████████████████]


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
#inputs = kobart_tokenizer(train.iloc[0,0], truncation=True, padding="longest", return_tensors="pt")
text = train.iloc[0,0]
raw_input_ids = kobart_tokenizer.encode(text)
input_ids = [kobart_tokenizer.bos_token_id] + raw_input_ids + [kobart_tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
kobart_tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

'웃긴게 20년동안... 바로 옆 가게 주차장에서 같이 그 감자탕집 주차도 했었는데 그 가게에서 이제 못대게해서; 졸지에... 졸지에...감자탕집 주차 없어짐  헐 의상하는 일 있었나? 뭐..아무래도 그 감자탕집이 장사 잘되니까 ᄏᄏᄏᄏᄏᄏᄏᄏᄏ 그렇다고 감자탕집을 안간다해서 그 가게는 안갈거같은데; 거기는..뭔 아 옆 가게 주차장에서 같이 그 감자탕집 주차도 했었는데 같이 그 감자탕집 주차도 했었는데 그 가게에서 이제 못대게해서; 졸지에... 감자탕집 주차 없어게해서; 졸지에...감자탕집 주차 없어짐  헐 의상하는 일 있었나? 뭐..아무래도 그 감자탕집이 장사 잘되니까 ᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏ 배아파서.. 아.. 그렇다고 감자탕집을 안간다해서 그 가게는 안갈거같은데; 거기는..뭔 능이버섯 오리전골? 뭐 이런거라 대중적이진 않잖아 아 점심으로 투머치.. 아 점심으로 투머치.. 아 점심으로 투머치.... 아머치.. 아음..감자탕집 장사 잘되니까 장사 잘되니까 ᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏ 배아파서.. 아 점심으로 투머치....뭔 능이버섯 오리전골? 뭐 이런거라 대중적이진 않잖아 아 점심으로 투머치...... 점심으로 투머치............감자탕집 주차 없어짐....감자탕집 주차 없어짐  헐 의상하는 일 있었나? 뭐..아무래도 그 감자탕집이 장사 잘되니까 ᄏᄏᄏᄏᄏᄏᄏᄏᄏ 배아파서.. 아.. 그렇다고 감자탕집을 안간다해서 그 가게는 안갈거같은데.. 아.. 그렇다고 감자탕집을 안간다해서 그 가게는 안갈거같은데; 거기는..뭔 능이버섯 오리전골? 뭐 이런거라 대중적이진 않잖아 아'

In [ ]:
#inputs = kobart_tokenizer(train.iloc[0,0], truncation=True, padding="longest", return_tensors="pt")
text = train.iloc[1,0]
raw_input_ids = kobart_tokenizer.encode(text)
input_ids = [kobart_tokenizer.bos_token_id] + raw_input_ids + [kobart_tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
kobart_tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

'아까 보니깐 로켓 프레시 잇드라 #@시스템#사진# 나도 밥 무거요 호앵 야 제주도는 로켓배송가능해? 글쎄 보고올게 어제 운전할때 보니딴 쿠팡 머 잇긴 하던데 운전하는데 폰을 왜 보나 보니 어제 운전할때 보니딴 쿠팡 머 잇긴 하던데 운전하는데 폰을 왜 보나 보니 #@시스템#사진# 일단 여긴 안돼 아니 운전하고 지나가는데 쿠팡 물류센터가 잇더라도 제주시 쪽에 그 로켓배송 말구 다른 택배로 천천히라도 와? ᄀᄅ쎄....??????? 나도 밥 무거요 호앵 야 제주도는 로켓배송가능해? 글쎄 보고올게 어제 운전할때 보니딴 쿠팡 머 잇긴 하던데 운전긴 하던데 운전하는데 폰을 왜 보나 보니 #@시스템#사진# 일단 여긴 안돼 아니 쿠팡 머 잇긴 하던데 운전하는데 폰을 왜 보나 보니 #@시스템#사진# 일단 여긴 안돼 아니 운전하고 지나가는데 쿠팡 물류센터가 잇더라도 제주시 쪽에 그 로켓배송 말구 다른 택배로 천천히라도 와? ᄀᄅ쎄....?? 천천히라도 와? ᄀᄅ쎄....?????? ᄀᄅ쎄....???? 나도 밥 무거요 호앵 야 제주도는 로켓배송가능해? 글쎄 보고올게 어제 운전할때 보니딴 쿠팡 머 잇긴 하던데 운전하는데 폰을 왜 보나 보니 #@시스템#사진#사진# 일단 여긴 안돼 아니 운전하고 지나가는데 쿠팡 물류센터가 잇더라도 일단 여긴 안돼 아니 운전하고 지나가는데 쿠팡 물류센터가 잇더라도 제주시 쪽에 그 로켓배송 말구 다른 택배로 천천히라도 와? ᄀᄅ쎄....? #@시스템#사진#사진# 나도 밥 무앵 야 제주도는 로켓배송가능해? 글쎄 보고올게 어제 운전할때 보니깐 로켓 배앵 야 제주도는 로켓배송가능해? 글쎄 보고올게 어제 운전할때 보니깐 로켓 프레시 잇드라 #@시스템#사진# 아까 보니깐 로켓 프레시 잇드라 #@시스템#사진# 나도 밥 무거요 호앵 야 제주도는 로켓배송가능해? #@시스템#사진# 나도 밥 무거요 호앵 야 제주도는 로켓배송가능해? 글쎄 보고올게 어제 운전할때 보니딴 쿠팡 머 잇긴 하던데 운전하는데 '

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# KOBART 모델과 토크나이저 로드
model_name = "skt/kobart"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 요약할 문장 입력
input_text = "한국어 문장을 요약하는 예제입니다."

# 문장을 토큰화하고 입력 인코딩
inputs = tokenizer(input_text, truncation=True, padding="longest", return_tensors="pt")

# 모델에 입력 전달하여 요약 생성
summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=100, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 요약 출력
print("Input:", input_text)
print("Summary:", summary)


In [ ]:
!nvidia-smi